# **NOTE**
This submission is only built for the *DRAFT* stage; therefore, the final report and results are not ready yet.

# Author
**Aryan Kukreja**

aryansk2@illinois.edu

UIN: 652936393

GitHub Repo: https://github.com/ABusyProgrammer/CS598-DLH

# Introduction

In the rapidly evolving field of healthcare analytics, Electronic Health Records (EHRs) have emerged as a vital resource. These records, which contain a wealth of patient information, hold the potential to significantly improve patient outcomes and optimize the allocation of healthcare resources. However, the inherent complexity of EHRs, characterized by high sparsity and irregular observations, presents a formidable challenge. Traditional time series analysis methods, designed for densely sampled data, often fall short when applied to EHRs. While state-of-the-art methods such as the various categories and implementations of neural network models and attention mechanisms have shown promising results, they come with their own set of limitations. These methods often necessitate substantial computational resources and involve truncating inputs, which can compromise the accuracy of the predictions.

Addressing these challenges, this paper introduces a groundbreaking approach with the Dual Event Time Transformer (DuETT). DuETT represents a significant departure from traditional methods, offering an innovative architecture that attends to both the time and event type dimensions of EHR data. This unique capability allows DuETT to transform sparse time series into a regular sequence, thereby enabling the application of larger and deeper neural networks. This transformation process, which is at the heart of DuETT’s innovation, effectively handles the irregularity and sparsity of EHR data. The effectiveness of DuETT is not just theoretical; it has been empirically demonstrated. DuETT outperforms state-of-the-art deep learning models on multiple downstream tasks using the MIMIC-IV and PhysioNet-2012 EHR datasets. By providing a robust and effective representation of EHR data, DuETT makes a significant contribution to the field of healthcare analytics. Its state-of-the-art performance and potential for practical applications in hospitals underscore the importance and relevance of this research in the ongoing efforts to leverage EHRs for improved healthcare outcomes.

Paper being analyzed: [DuETT: Dual Event Time Transformer for Electronic Health
Records](https://arxiv.org/pdf/2304.13017.pdf)

# Scope of Reproducibility:
The following are some of the key hypothesis that this paper looks to test:
1. **Capture EHR Structure**: If DuETT attends over both time and event dimensions of EHR data, then it can produce robust representations that capture the structure of EHR data.
2. **Handle Sparsity and Irregularity**: If DuETT transforms sparse and irregularly sampled time series into regular sequences with fixed length, then it can reduce computational complexity and handle the sparsity and irregularity of EHR data.
3. **Improve Model Performance**: If DuETT is applied to multiple downstream tasks using the MIMIC-IV and PhysioNet-2012 EHR datasets, then it can outperform state-of-the-art deep learning models.
4. **Leverage Self-Supervised Learning**: If DuETT utilizes self-supervised prediction tasks for model pre-training, then it can enable the training of larger models with limited labeled data.

These hypotheses form the basis of the paper’s investigation into the effectiveness of the DuETT architecture for modeling EHR data. Each hypothesis is designed to test a specific aspect of DuETT’s capabilities and its potential advantages over existing methods.

# Implementation

The following is an attempt to replicate the paper I have selected.

## Imports

The following is where we will import all the libraries in general, to keep the code clean throughout the rest of the file.

In [ ]:
!pip numpy==1.21.5
!pip install pytorch-lightning==1.6.1
!pip install torch==1.13.1
!pip install torchaudio==0.13.1
!pip install torchvision==0.14.1
!pip install x-transformers==1.5.3
!pip install torchtime==0.5.1
!pip install torchmetrics==0.8.0

In [ ]:
# import  packages you need
import torch
from torch.utils.data import Dataset, DataLoader
from torchtime.data import PhysioNet2012
from multiprocessing import Manager
import pytorch_lightning as pl

import torchmetrics
import torch.nn.functional as F
import torch.nn as nn
import pytorch_lightning as pl
import numpy as np
import x_transformers

import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')
import pytorch_lightning as pl
import argparse

## Data Pre-Processing Utilities

Prior to conducting the actual training itself, the data processing pipeline for a machine learning model requires the use of a data pre-processing to ensure that the data that will be used for training is clean and refined to produce valid results. This paper performs all these steps as part of the [physionet.py](https://github.com/layer6ai-labs/DuETT/blob/master/physionet.py) file, which had the `PhysioNetDataModule` and `PhysioNetDataset` classes defined. Each of them would do the following:
1. `PhysioNetDataset`:
   1. **Initialization**: Sets up the dataset split (train, validation, test), number of timesteps, and whether to use a temporary cache.
   1. **Setup**: Prepares the data by loading it from the PhysioNet2012 source and calculating statistics like mean, standard deviation, max, and min for normalization.
   1. **Data Retrieval**: Implements methods to get the length of the dataset and retrieve individual items, which includes preprocessing and normalizing the time-series and static data.
2. `PhysioNetDataModule`:
   1. **Initialization**: Configures the batch size, number of workers, and other DataLoader parameters.
   1. **Setup**: Calls the setup method of the PhysioNetDataset instances for different splits.
   1. **Data Loaders**: Provides DataLoader instances for training, validation, and testing phases, which use the collate_into_seqs function to batch the data.

As part of my attempts to replicate this paper, I have attempted to simplify the overall structure of the data pre-processing code into a single class for easier understanding, and have made some optimizations with reference to the rest of the code as well.

In [ ]:
class PhysioNetDataModule(pl.LightningDataModule):
    def __init__(self, split_name, n_timesteps=32, use_temp_cache=False, batch_size=8, num_workers=1, prefetch_factor=2, **kwargs):
        super().__init__()
        self.split_name = split_name
        self.n_timesteps = n_timesteps
        self.temp_cache = Manager().dict() if use_temp_cache else None
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.prefetch_factor = prefetch_factor
        self.setup()

    def setup(self, stage=None):
        # Initialize the dataset with the specified split
        tt_data = PhysioNet2012(self.split_name, train_prop=0.7, val_prop=0.15, time=False, seed=0)
        self.X = tt_data.X
        self.y = tt_data.y
        self.calculate_statistics()

    def calculate_statistics(self):
        # Calculate statistics for normalization
        self.means = []
        self.stds = []
        self.maxes = []
        self.mins = []
        for i in range(self.X.shape[2]):
            vals = self.X[:,:,i].flatten()
            vals = vals[~torch.isnan(vals)]
            self.means.append(vals.mean())
            self.stds.append(vals.std())
            self.maxes.append(vals.max())
            self.mins.append(vals.min())

    def __len__(self):
        # Return the number of samples in the dataset
        return self.X.shape[0]

    def __getitem__(self, i):
        # Retrieve a sample and its label, applying normalization
        if self.temp_cache is not None and i in self.temp_cache:
            return self.temp_cache[i]
        ins = self.X[i, ~torch.isnan(self.X[i,:,0]), :]
        x, y = self.process_sample(ins)
        if self.temp_cache is not None:
            self.temp_cache[i] = (x, y)
        return x, y

    def process_sample(self, ins):
        # Process a single sample from the dataset
        time = ins[:,0] / 60 / 24
        x_static, x_ts, bin_ends = self.create_tensors(time)
        self.populate_tensors(ins, time, x_static, x_ts, bin_ends)
        y = self.y[i,0]
        return (x_ts, x_static, bin_ends), y

    def create_tensors(self, time):
        # Create tensors for time-series and static features
        x_static = torch.zeros(self.d_static_num())
        x_ts = torch.zeros((self.n_timesteps, self.d_time_series_num()*2))
        bin_ends = torch.arange(1, self.n_timesteps+1) / self.n_timesteps * time[-1]
        return x_static, x_ts, bin_ends

    def populate_tensors(self, ins, time, x_static, x_ts, bin_ends):
        # Populate the tensors with normalized data
        for i_t, t in enumerate(time):
            bin = self.n_timesteps - 1 if t == time[-1] else int(t / time[-1] * self.n_timesteps)
            for i_ts in range(1,37):
                x_i = ins[i_t,i_ts]
                if not torch.isnan(x_i).item():
                    x_ts[bin, i_ts-1] = (x_i - self.means[i_ts])/(self.stds[i_ts] + 1e-7)
                    x_ts[bin, i_ts-1+self.d_time_series_num()] += 1
        for i_tab in range(37,45):
            x_i = ins[0, i_tab]
            x_i = (x_i - self.means[i_tab])/(self.stds[i_tab] + 1e-7)
            x_static[i_tab-37] = x_i.nan_to_num(0.)

    def d_static_num(self):
        # Return the number of static features
        return 8

    def d_time_series_num(self):
        # Return the number of time-series features
        return 36

    def d_target(self):
        # Return the dimension of the target variable
        return 1

    def pos_frac(self):
        # Return the fraction of positive samples
        return self.y.mean().numpy()

    def collate_into_seqs(self, batch):
        # Collate a batch of samples into sequences
        xs, ys = zip(*batch)
        return zip(*xs), ys

    def train_dataloader(self):
        # Return a DataLoader for the training set
        return DataLoader(self, shuffle=True, batch_size=self.batch_size, prefetch_factor=self.prefetch_factor, collate_fn=self.collate_into_seqs, num_workers=self.num_workers)

    def val_dataloader(self):
        # Return a DataLoader for the validation set
        return DataLoader(self, batch_size=self.batch_size, prefetch_factor=self.prefetch_factor, collate_fn=self.collate_into_seqs, num_workers=self.num_workers)

    def test_dataloader(self):
        # Return a DataLoader for the test set
        return DataLoader(self, batch_size=self.batch_size, prefetch_factor=self.prefetch_factor, collate_fn=self.collate_into_seqs, num_workers=self.num_workers)


##   Model
Below is the (***in-progress***) implementation of the model for the training.

### Batch Normalization
Batch normalization is a technique used to increase the stability of a neural network. It normalizes the output of a previous activation layer by subtracting the batch mean and dividing by the batch standard deviation. Here, it s designed to apply this normalization along the last dimension of the input tensor. This can be particularly useful when dealing with multi-dimensional inputs where you want to apply normalization not over the entire batch, but separately for each feature in the last dimension.

In [ ]:
# Custom Batch Normalization for the last dimension of a tensor
class BatchNormLastDim(nn.Module):
    def __init__(self, d, **kwargs):
        super().__init__()
        self.batch_norm = nn.BatchNorm1d(d, **kwargs)

    # Apply batch normalization to the last dimension
    def forward(self, x):
        if x.ndim == 2:
            return self.batch_norm(x)
        elif x.ndim == 3:
            return self.batch_norm(x.transpose(1, 2)).transpose(1, 2)
        else:
            raise NotImplementedError("Not implemented in the paper... Anything I can do here?")

### Multi-Layer Perceptron Component
This paper relies on a simple implementation of a simple multi-layer perceptron that is used when building the overall neural network architecture. Due to its simple implementation, its been kept largely the same as in the original paper. (Might change this later).

In [ ]:
# Simple MLP (Multi-Layer Perceptron) with customizable layers and options
def simple_mlp(d_in, d_out, n_hidden, d_hidden, final_activation=False, input_batch_norm=False, hidden_batch_norm=False, dropout=0., activation=nn.ReLU):
    layers = []
    # Input batch normalization layer
    if input_batch_norm:
        layers.append(BatchNormLastDim(d_in))
    # Input linear layer
    layers.append(nn.Linear(d_in, d_hidden if n_hidden > 0 else d_out))
    # Hidden layers
    for _ in range(n_hidden):
        layers.extend([
            activation(),
            nn.Dropout(dropout),
            BatchNormLastDim(d_hidden) if hidden_batch_norm else nn.Identity(),
            nn.Linear(d_hidden, d_hidden)
        ])
    # Final linear layer
    layers.append(nn.Linear(d_hidden if n_hidden > 0 else d_in, d_out))
    # Final activation layer
    if final_activation:
        layers.append(activation())
    return nn.Sequential(*layers)

### Model Pre-Training and Fine-Tuning
These are helper methods. The `pretrain_model` method is used to initialize a model with a given set of parameters for pre-training. The `fine_tune_model` method, on the other hand, is called within a loop (later in this code) to iteratively fine-tune the model.

In [ ]:
# Model pretraining function
def pretrain_model(d_static_num, d_time_series_num, d_target, **kwargs):
    return Model(d_static_num, d_time_series_num, d_target, **kwargs)

# Model fine-tuning function
def fine_tune_model(ckpt_path, **kwargs):
    return Model.load_from_checkpoint(ckpt_path, pretrain=False, aug_noise=0., aug_mask=0.5, transformer_dropout=0.5, lr=1.e-4, weight_decay=1.e-5, fusion_method='rep_token', **kwargs)

### Core Model
This is the main model class that defines the key training functions/methods such as `forward()`. It is a PyTorch Lightning module designed for training and evaluating a machine learning model for time-series data analysis that is being conducted here to eveluate the DuETT approach.

In [ ]:
class Model(pl.LightningModule):
    # Model initialization
    def __init__(self, d_static_num, d_time_series_num, d_target, lr=3.e-4, weight_decay=1.e-1, glu=False, scalenorm=True, n_hidden_mlp_embedding=1, d_hidden_mlp_embedding=64, d_embedding=24, d_feedforward=512, max_len=48, n_transformer_head=2, n_duett_layers=2, d_hidden_tab_encoder=128, n_hidden_tab_encoder=1, norm_first=True, fusion_method='masked_embed', n_hidden_head=1, d_hidden_head=64, aug_noise=0., aug_mask=0., pretrain=True, pretrain_masked_steps=1, pretrain_n_hidden=0, pretrain_d_hidden=64, pretrain_dropout=0.5, pretrain_value=True, pretrain_presence=True, pretrain_presence_weight=0.2, predict_events=True, transformer_dropout=0., pos_frac=None, freeze_encoder=False, seed=0, save_representation=None, masked_transform_timesteps=32, **kwargs):
        super().__init__()
        # Model configuration parameters
        self.lr = lr
        self.weight_decay = weight_decay
        self.d_time_series_num = d_time_series_num
        self.d_target = d_target
        self.d_embedding = d_embedding
        self.max_len = max_len
        self.pretrain = pretrain
        self.pretrain_masked_steps = pretrain_masked_steps
        self.pretrain_dropout = pretrain_dropout
        self.freeze_encoder = freeze_encoder
        self.set_pos_frac(pos_frac)
        self.rng = np.random.default_rng(seed)
        self.aug_noise = aug_noise
        self.aug_mask = aug_mask
        self.fusion_method = fusion_method
        self.pretrain_presence = pretrain_presence
        self.pretrain_presence_weight = pretrain_presence_weight
        self.predict_events = predict_events
        self.masked_transform_timesteps = masked_transform_timesteps
        self.pretrain_value = pretrain_value
        self.save_representation = save_representation
        # Special embeddings for different purposes
        self.special_embeddings = nn.Embedding(8, d_embedding)
        # Embedding layers for time series features
        self.embedding_layers = nn.ModuleList([
            simple_mlp(2, d_embedding, n_hidden_mlp_embedding, d_hidden_mlp_embedding, hidden_batch_norm=True)
            for _ in range(d_time_series_num)
        ])
        # Embedding for the number of observations
        self.n_obs_embedding = nn.Embedding(16, 1)
        # Feedforward dimensions
        if d_feedforward is None:
            d_feedforward = d_embedding * 4
        et_dim = d_embedding * (masked_transform_timesteps + 1)
        tt_dim = d_embedding * (d_time_series_num + 1)
        # Transformer layers for event and time features
        self.event_transformers = nn.ModuleList([
            x_transformers.Encoder(dim=et_dim, depth=1, heads=n_transformer_head, pre_norm=norm_first, use_scalenorm=scalenorm, attn_dim_head=d_embedding // n_transformer_head, ff_glu=glu, ff_mult=d_feedforward / et_dim, attn_dropout=transformer_dropout, ff_dropout=transformer_dropout)
            for _ in range(n_duett_layers)
        ])
        self.full_event_embedding = nn.Embedding(d_time_series_num + 1, et_dim)
        self.time_transformers = nn.ModuleList([
            x_transformers.Encoder(dim=tt_dim, depth=1, heads=n_transformer_head, pre_norm=norm_first, use_scalenorm=scalenorm, attn_dim_head=d_embedding // n_transformer_head, ff_glu=glu, ff_mult=d_feedforward / tt_dim, attn_dropout=transformer_dropout, ff_dropout=transformer_dropout)
            for _ in range(n_duett_layers)
        ])
        self.full_time_embedding = self.cve(batch_norm=True, d_embedding=tt_dim)
        self.full_rep_embedding = nn.Embedding(tt_dim, 1)
        # Representation dimension
        d_representation = d_embedding * (d_time_series_num + 1)  # time_series + static
        # Prediction head
        self.head = simple_mlp(d_representation, d_target, n_hidden_head, d_hidden_head, hidden_batch_norm=True, final_activation=False, activation=nn.ReLU)
        # Pretraining projection layers
        self.pretrain_value_proj = simple_mlp(d_representation, d_time_series_num, pretrain_n_hidden, pretrain_d_hidden, hidden_batch_norm=True)
        if self.pretrain_presence:
            self.pretrain_presence_proj = simple_mlp(d_representation, d_time_series_num, pretrain_n_hidden, pretrain_d_hidden, hidden_batch_norm=True)
        if self.predict_events:
            self.predict_events_proj = simple_mlp(et_dim, masked_transform_timesteps, pretrain_n_hidden, pretrain_d_hidden, hidden_batch_norm=True)
            if self.pretrain_presence:
                self.predict_events_presence_proj = simple_mlp(et_dim, masked_transform_timesteps, pretrain_n_hidden, pretrain_d_hidden, hidden_batch_norm=True)
        # Encoder for static features
        self.tab_encoder = simple_mlp(d_static_num, d_embedding, n_hidden_tab_encoder, d_hidden_tab_encoder, hidden_batch_norm=True)
        # Loss functions
        self.pretrain_loss = F.mse_loss
        self.loss_function = F.binary_cross_entropy_with_logits
        self.pretrain_presence_loss = F.binary_cross_entropy_with_logits
        # Metrics
        num_classes = None if d_target == 1 else d_target
        self.train_auroc = torchmetrics.AUROC(num_classes=num_classes)
        self.val_auroc = torchmetrics.AUROC(num_classes=num_classes)
        self.train_ap = torchmetrics.AveragePrecision(num_classes=num_classes)
        self.val_ap = torchmetrics.AveragePrecision(num_classes=num_classes)
        self.test_auroc = torchmetrics.AUROC(num_classes=num_classes)
        self.test_ap = torchmetrics.AveragePrecision(num_classes=num_classes)

    # Set the positive fraction for weighted loss calculation
    def set_pos_frac(self, pos_frac):
        if type(pos_frac) == list:
            pos_frac = torch.tensor(pos_frac, device=torch.device('cuda'))
        self.pos_frac = pos_frac
        if pos_frac is not None:
            self.pos_weight = 1 / (2 * pos_frac)
            self.neg_weight = 1 / (2 * (1 - pos_frac))

    # Custom value embedding function
    def cve(self, d_embedding=None, batch_norm=False):
        if d_embedding is None:
            d_embedding = self.d_embedding
        d_hidden = int(np.sqrt(d_embedding))
        layers = [nn.Linear(1, d_hidden), nn.Tanh()]
        if batch_norm:
            layers.append(BatchNormLastDim(d_hidden))
        layers.append(nn.Linear(d_hidden, d_embedding))
        return nn.Sequential(*layers)

    # Convert features to model input
    def feats_to_input(self, x, batch_size, limits=None):
        # Unpack the input tuple
        xs_ts, xs_static, times = x
        xs_ts = list(xs_ts)

        # Process each time series feature
        for i, f in enumerate(xs_ts):
            n_vars = f.shape[1] // 2  # Number of variables in the feature

            # Truncate the feature to the maximum length if necessary
            if f.shape[0] > self.max_len:
                f = f[-self.max_len:]
                times[i] = times[i][-self.max_len:]

            # Apply noise augmentation during training
            if self.training and self.aug_noise > 0 and not self.pretrain:
                f[:, :n_vars] += self.aug_noise * torch.randn_like(f[:, :n_vars]) * f[:, n_vars:]

            # Add a new dimension for the mask
            f = torch.cat((f, torch.zeros_like(f[:, :1])), dim=1)

            # Apply mask augmentation during training
            if self.training and self.aug_mask > 0 and not self.pretrain:
                mask = torch.rand(f.shape[0]) < self.aug_mask
                f[mask, :] = 0.
                f[mask, -1] = 1.

            xs_ts[i] = f

        # Determine the maximum number of timesteps across all features
        n_timesteps = [len(ts) for ts in times]
        pad_to = max(n_timesteps)

        # Pad the time series and static features to have the same length
        xs_ts = torch.stack([F.pad(t, (0, 0, 0, pad_to - t.shape[0])) for t in xs_ts]).to(self.device)
        xs_times = torch.stack([F.pad(t, (0, pad_to - t.shape[0])) for t in times]).to(self.device)
        xs_static = torch.stack(xs_static).to(self.device)

        # Apply noise augmentation to static features during training
        if self.training and self.aug_noise > 0 and not self.pretrain:
            xs_static += self.aug_noise * torch.randn_like(xs_static)

        return xs_static, xs_ts, xs_times, n_timesteps

    def pretrain_prep_batch(self, x, batch_size):
        # Prepare the batch for pretraining by augmenting and masking the data
        xs_static, xs_ts, xs_times, n_timesteps = self.feats_to_input(x, batch_size)
        n_steps = xs_ts.shape[1]
        n_vars = (xs_ts.shape[2] - 1) // 2

        # Initialize lists to hold targets for pretraining tasks
        y_ts = []
        y_ts_n_obs = []
        y_events = []
        y_events_mask = []

        # Clone the time series features to apply masking
        xs_ts_clipped = xs_ts.clone()

        # Iterate over each sample in the batch
        for batch_i, n in enumerate(n_timesteps):
            # Determine indices to mask based on the number of timesteps and pretraining configuration
            mask_i = self.rng.choice(np.arange(n), size=min(self.pretrain_masked_steps, n), replace=False)

            # Store the original values and observation flags for the masked timesteps
            y_ts.append(xs_ts[batch_i, mask_i, :n_vars])
            y_ts_n_obs.append(xs_ts[batch_i, mask_i, n_vars:2*n_vars])

            # Apply masking to the cloned features
            xs_ts_clipped[batch_i, mask_i, :] = 0.
            xs_ts_clipped[batch_i, mask_i, -1] = 1.

            # If event prediction is enabled, prepare event targets and masks
            if self.predict_events:
                event_mask_i = self.rng.choice(np.arange(0, self.d_time_series_num))
                y_events.append(xs_ts[batch_i, :, event_mask_i])
                y_events_mask.append(xs_ts[batch_i, :, event_mask_i + n_vars].clip(0, 1))
                xs_ts_clipped[batch_i, :, event_mask_i] = 0
                xs_ts_clipped[batch_i, :, event_mask_i + n_vars] = -1

        # Convert lists to tensors
        y_ts = torch.stack(y_ts)
        y_ts_n_obs = torch.stack(y_ts_n_obs)
        y_ts_masks = y_ts_n_obs.clip(0, 1)

        # Handle event targets and masks if applicable
        if len(y_events) > 0:
            y_events = torch.stack(y_events)
            y_events_mask = torch.stack(y_events_mask)

        # Apply dropout to the masked features if configured
        if self.pretrain_dropout > 0:
            keep = self.rng.random((batch_size, n_vars)) > self.pretrain_dropout
            keep = torch.tensor(keep, device=xs_ts.device)
            keep = torch.cat((keep.tile(1, 2), torch.ones((batch_size, 1), device=keep.device)), dim=1)
            xs_ts_clipped *= torch.logical_or(keep.unsqueeze(1), xs_ts_clipped == -1)

        # Return the prepared batch along with the targets and masks
        return (xs_static, xs_ts_clipped, xs_times, n_timesteps), y_ts, y_ts_masks, y_events, y_events_mask

    def forward(self, x, pretrain=False, representation=False):
        """
        Forward pass of the model.

        Args:
            x (tuple): Input data consisting of features and times.
            pretrain (bool): Flag to indicate if the model is in pretraining mode.
            representation (bool): Flag to indicate if the model should return representations.

        Returns:
            torch.Tensor: The output of the model which could be predictions, representations, or pretraining projections.
        """
        # Unpack input data
        xs_static, xs_feats, xs_times, n_timesteps = x
        n_vars = xs_feats.shape[2] // 2  # Number of variables based on feature shape

        # Prepare masks and embeddings for event prediction if enabled
        if self.predict_events:
            event_mask_inds = xs_feats[:, :, n_vars:n_vars*2] == -1
            event_mask_inds = torch.cat((event_mask_inds, torch.zeros(xs_feats.shape[:2] + (1,), device=xs_feats.device, dtype=torch.bool)), dim=2)
            event_mask_inds = torch.cat((event_mask_inds, event_mask_inds[:, :1, :]), dim=1)
            n_obs_inds = xs_feats[:, :, n_vars:n_vars*2].to(int).clip(0, self.n_obs_embedding.num_embeddings - 1)
            xs_feats[:, :, n_vars:n_vars*2] = self.n_obs_embedding(n_obs_inds).squeeze(-1)

        # Process features for embedding
        embedding_layer_input = torch.empty(xs_feats.shape[:-1] + (n_vars, 2), dtype=xs_feats.dtype, device=xs_feats.device)
        embedding_layer_input[:, :, :, 0] = xs_feats[:, :, :n_vars]
        embedding_layer_input[:, :, :, 1] = xs_feats[:, :, n_vars:n_vars*2]

        # Embedding and transformation operations
        psi = torch.zeros((xs_feats.shape[0], xs_feats.shape[1]+1, n_vars+1, self.d_embedding), dtype=xs_feats.dtype, device=xs_feats.device)
        for i, el in enumerate(self.embedding_layers):
            psi[:, :-1, i, :] = el(embedding_layer_input[:, :, i, :])
        psi[:,:-1,-1,:] = self.tab_encoder(xs_static).unsqueeze(1)
        psi[:,-1,:,:] = self.special_embeddings(self.REPRESENTATION_EMBEDDING_KEY.to(self.device)).unsqueeze(0).unsqueeze(1)
        mask_inds = torch.cat((xs_feats[:,:,-1] == 1, torch.zeros((xs_feats.shape[0], 1), device=xs_feats.device, dtype=torch.bool)), dim=1)
        psi[mask_inds, :, :] = self.special_embeddings(self.MASKED_EMBEDDING_KEY.to(self.device))
        if self.predict_events:
            psi[event_mask_inds, :] = self.special_embeddings(self.MASKED_EMBEDDING_KEY.to(self.device))

        # batch, time step, full embedding
        time_embeddings = self.full_time_embedding(xs_times.unsqueeze(2))
        time_embeddings = torch.cat((time_embeddings,
            self.full_rep_embedding.weight.T.unsqueeze(0).expand(xs_feats.shape[0],-1,-1)),
            dim=1)
        for layer_i, (event_transformer, time_transformer) in enumerate(zip(self.event_transformers, self.time_transformers)):
            et_out_shape = (psi.shape[0], psi.shape[2], psi.shape[1], psi.shape[3])
            embeddings = psi.transpose(1,2).flatten(2) + self.full_event_embedding.weight.unsqueeze(0)
            event_outs = event_transformer(embeddings).view(et_out_shape).transpose(1,2)
            tt_out_shape = event_outs.shape
            embeddings = event_outs.flatten(2) + time_embeddings
            psi = time_transformer(embeddings).view(tt_out_shape)
        transformed = psi.flatten(2)

        if self.fusion_method == 'rep_token':
            z_ts = transformed[:,-1,:]
        elif self.fusion_method == 'masked_embed':
            if self.pretrain_masked_steps > 1:
                masked_ind = F.pad(xs_feats[:,:,-1] > 0, (0,1), value=False)
                z_ts = []
                for i in range(transformed.shape[0]):
                    z_ts.append(F.pad(transformed[i, masked_ind[i],:], (0,0,0,self.pretrain_masked_steps-masked_ind[i].sum()), value=0.))
                z_ts = torch.stack(z_ts) # batch size x pretrain_masked_steps x d_embedding
            else:
                masked_ind = xs_feats[:,:,-1:]
                z_ts = []
                for i in range(transformed.shape[0]):
                    z_ts.append(transformed[i, torch.nonzero(masked_ind[i].squeeze()==1),:])
                z_ts = torch.cat(z_ts, dim=0).squeeze()
        elif self.fusion_method == 'averaging':
            z_ts = torch.mean(transformed[:,:-1,:], dim=1)

        # Return the final output based on the mode
        z = z_ts
        if representation:
            return z
        if pretrain:
            rep_token_head = torch.tile(transformed[:,0,:].unsqueeze(1), (1, self.masked_transform_timesteps, 1))
            y_hat_presence = self.pretrain_presence_proj(z).squeeze() if self.pretrain_presence else None
            y_hat_value = self.pretrain_value_proj(z).squeeze(1) if self.pretrain_value else None
            z_events = []
            y_hat_events, y_hat_events_presence = None, None
            if self.predict_events:
                for i in range(event_mask_inds.shape[0]):
                    z_events.append(psi[i][event_mask_inds[i].nonzero(as_tuple=True)].flatten())
                z_events = torch.stack(z_events)
                y_hat_events = self.predict_events_proj(z_events).squeeze()
                y_hat_events_presence = self.predict_events_presence_proj(z_events).squeeze() if self.pretrain_presence else None
            return y_hat_value, y_hat_presence, y_hat_events, y_hat_events_presence
        else:
            out = self.head(z).squeeze(1)
            if self.save_representation:
                return out, z
            else:
                return out

    def configure_optimizers(self):
        optimizers = [torch.optim.AdamW([p for l in self.modules() for p in l.parameters()],
                lr=self.lr, weight_decay=self.weight_decay)]
        return optimizers

    def training_step(self, batch, batch_idx):
        # Unpack the batch data
        x, y = batch
        y = torch.tensor(y, dtype=torch.float64, device=self.device)
        batch_size = y.shape[0]

        # Check if the model is in pretraining mode
        if self.pretrain:
            # Prepare the batch for pretraining
            x_pretrain, y, mask, y_events, y_events_mask = self.pretrain_prep_batch(x, batch_size)
            # Forward pass for pretraining
            y_hat_value, y_hat_presence, y_hat_events, y_hat_events_presence = self.forward(x_pretrain, pretrain=True)

            # Initialize the loss
            loss = 0
            # Calculate the value prediction loss if enabled
            if self.pretrain_value:
                loss += self.pretrain_loss(y_hat_value * mask, y * mask).mean()
            # Calculate the presence prediction loss if enabled
            if self.pretrain_presence:
                loss += self.pretrain_presence_loss(y_hat_presence, mask).mean() * self.pretrain_presence_weight
            # Calculate the event prediction loss if enabled
            if self.predict_events:
                loss += self.pretrain_loss(y_hat_events * y_events_mask, y_events * y_events_mask).mean()
        else:
            # Forward pass for standard training
            y_hat = self.forward(self.feats_to_input(x, batch_size))
            # Calculate the loss with optional class weighting
            if self.pos_frac is not None:
                weight = torch.where(y > 0, self.pos_weight, self.neg_weight)
                loss = self.loss_function(y_hat, y, weight)
            else:
                loss = self.loss_function(y_hat, y)

        # Update metrics and log the training loss
        self.train_auroc.update(y_hat, y.to(int))
        self.train_ap.update(y_hat, y.to(int))
        self.log('train_loss', loss, sync_dist=True)

        return loss

        def validation_step(self, batch, batch_idx):
            # Unpack the batch data
            x, y = batch
            y = torch.tensor(y, dtype=torch.float64, device=self.device)
            batch_size = y.shape[0]

            # Check if the model is in pretraining mode
            if self.pretrain:
                # Prepare the batch for pretraining
                x_pretrain, y, mask, y_events, y_events_mask = self.pretrain_prep_batch(x, batch_size)
                # Forward pass for pretraining
                y_hat_value, y_hat_presence, y_hat_events, y_hat_events_presence = self.forward(x_pretrain, pretrain=True)

                # Initialize the loss
                loss = 0
                # Calculate the value prediction loss if pretraining value prediction is enabled
                if self.pretrain_value:
                    loss += self.calculate_pretrain_loss(y_hat_value, y, mask)
                # Calculate the presence prediction loss if pretraining presence prediction is enabled
                if self.pretrain_presence:
                    presence_loss = self.calculate_pretrain_loss(y_hat_presence, mask, presence_weight=self.pretrain_presence_weight)
                    loss += presence_loss
                # Calculate the event prediction loss if event prediction is enabled
                if self.predict_events:
                    event_loss = self.calculate_pretrain_loss(y_hat_events, y_events, y_events_mask)
                    loss += event_loss

                # Log validation losses
                self.log('val_next_loss', loss, on_epoch=True, sync_dist=True, rank_zero_only=True)
                if self.pretrain_presence:
                    self.log('val_presence_loss', presence_loss, on_epoch=True, sync_dist=True, rank_zero_only=True)
                if self.predict_events:
                    self.log('val_event_loss', event_loss, on_epoch=True, sync_dist=True, rank_zero_only=True)

            else:
                # Forward pass for validation
                y_hat = self.forward(self.feats_to_input(x, batch_size))
                # Calculate and log the validation loss
                loss = self.calculate_loss(y_hat, y)
                self.log('val_loss', loss, on_epoch=True, sync_dist=True, prog_bar=True, rank_zero_only=True)

                # Update and log AUROC and Average Precision metrics
                self.val_auroc.update(y_hat, y.to(int).to(self.device))
                self.val_ap.update(y_hat, y.to(int).to(self.device))
                self.log_metrics('val', self.val_auroc, self.val_ap)

            return loss

    def training_epoch_end(self, training_step_outputs):
        if not self.pretrain:
            self.log('train_auroc', self.train_auroc, sync_dist=True, rank_zero_only=True)
            self.log('train_ap', self.train_ap, sync_dist=True, rank_zero_only=True)

    def validation_epoch_end(self, validation_step_outputs):
        if not self.pretrain:
            print("val_auroc", self.val_auroc.compute(), "val_ap", self.val_ap.compute())


    def test_step(self, batch, batch_idx):
        # Unpack the batch data
        x, y = batch
        y = torch.tensor(y, dtype=torch.float64, device=self.device)
        batch_size = y.shape[0]

        # Check if we need to save representations
        if self.save_representation:
            # Forward pass to get predictions and representations
            y_hat, z = self.forward(self.feats_to_input(x, batch_size))
            print("saving representations...")
            # Save the representations to a file
            with open(self.save_representation, 'ab') as f:
                # Check if the target is multi-dimensional and save accordingly
                if y.ndim == 1:
                    np.savetxt(f, np.concatenate([z.cpu(), y.unsqueeze(1).cpu()], axis=1))
                else:
                    np.savetxt(f, np.concatenate([z.cpu(), y.cpu()], axis=1))
        else:
            # Forward pass to get predictions
            y_hat = self.forward(self.feats_to_input(x, batch_size))

        # Calculate loss
        if self.pos_frac is not None:
            # Apply different weights for positive and negative samples
            weight = torch.where(y > 0, self.pos_weight, self.neg_weight)
            loss = self.loss_function(y_hat, y, weight)
        else:
            loss = self.loss_function(y_hat, y)

        # Log the test loss
        self.log('test_loss', loss, on_epoch=True, sync_dist=True, rank_zero_only=True)

        # Update metrics
        self.test_auroc.update(y_hat, y.to(int).to(self.device))
        self.log('test_auroc', self.test_auroc, on_epoch=True, sync_dist=True, rank_zero_only=True)
        self.test_ap.update(y_hat, y.to(int).to(self.device))
        self.log('test_ap', self.test_ap, on_epoch=True, sync_dist=True, rank_zero_only=True)

        # Return the loss and metrics for further processing if needed
        return loss, self.test_auroc, self.test_ap

    def on_load_checkpoint(self, checkpoint):
        # Load state dict from the checkpoint
        state_dict = checkpoint["state_dict"]
        model_state_dict = self.state_dict()

        # Initialize a flag to track changes
        is_changed = False

        # Update model's state dict with checkpoint's state dict
        for k in model_state_dict:
            if k not in state_dict:
                state_dict[k] = model_state_dict[k]
                is_changed = True

        # Adjust the loaded state dict to match the model's state dict
        for k in list(state_dict.keys()):
            if k in model_state_dict:
                # Skip loading parameters with mismatched shapes, particularly in the 'head' layer
                if k.startswith('head') and state_dict[k].shape != model_state_dict[k].shape:
                    print(f"Skip loading parameter: {k}, "
                          f"required shape: {model_state_dict[k].shape}, "
                          f"loaded shape: {state_dict[k].shape}")
                    state_dict[k] = model_state_dict[k]
                    is_changed = True
            else:
                # Remove parameters not present in the model's state dict
                print(f"Dropping parameter {k}")
                del state_dict[k]
                is_changed = True

        # If changes were made, clear optimizer states to avoid conflicts
        if is_changed:
            checkpoint.pop("optimizer_states", None)

        # If the encoder is frozen, ensure its parameters are not updated
        if self.freeze_encoder:
            self.freeze()

    def freeze(self):
        print('Freezing')
        for n, w in self.named_parameters():
            if "head" not in n:
                w.requires_grad = False
            else:
                print("Skip freezing:", n)


### Learning-Rate Adjustment
This is an adjustment that is applied in the initial phase of the training (also known as the warm-up training phase). The task here is to gradually increase the learning rate over a provided number of steps. This is to prevent massive weight updates from the get-go of the training that can lead to divergence or poor convergence.

In [ ]:
import torch

class WarmUpCallback(torch.optim.lr_scheduler._LRScheduler):
    """
    A callback for linearly warming up the learning rate over a specified number of steps.
    Optionally applies an inverse square root decay after warmup.
    """
    def __init__(self, optimizer, warmup_steps, base_lr=None, invsqrt=True, decay=None, last_epoch=-1):
        # Initialize the parent class
        super().__init__(optimizer, last_epoch)

        # Store the warmup steps and base learning rate
        self.warmup_steps = warmup_steps
        self.base_lr = base_lr if base_lr is not None else [group['lr'] for group in optimizer.param_groups]
        self.invsqrt = invsqrt  # Whether to apply inverse square root decay
        self.decay = decay if decay is not None else warmup_steps  # Decay steps

        # Initialize the step counter
        self.steps = 0

    def set_lr(self, optimizer, lr):
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

    def on_train_batch_start(self, trainer, model, batch, batch_idx):
        # Retrieve the optimizer(s) from the model
        optimizers = model.optimizers()

        # Check if the warmup phase is still ongoing
        if self.state['steps'] < self.warmup_steps:
            # If base learning rate is not set, infer it from the optimizer(s)
            if self.state['base_lr'] is None:
                self.state['base_lr'] = [o.param_groups[0]['lr'] for o in optimizers] if isinstance(optimizers, list) else optimizers.param_groups[0]['lr']

            # Calculate the new learning rate based on the current step
            new_lr = self.state['steps'] / self.warmup_steps * self.state['base_lr']

            # Apply the new learning rate to the optimizer(s)
            if isinstance(optimizers, list):
                for opt, base_lr in zip(optimizers, self.state['base_lr']):
                    self.set_lr(opt, new_lr)
            else:
                self.set_lr(optimizers, new_lr)

        # If warmup is complete and inverse square root scaling is enabled
        elif self.invsqrt:
            # Calculate the decayed learning rate
            decayed_lr = self.state['base_lr'] * (self.decay / (self.state['steps'] - self.warmup_steps + self.decay)) ** 0.5

            # Apply the decayed learning rate to the optimizer(s)
            if isinstance(optimizers, list):
                for opt, base_lr in zip(optimizers, self.state['base_lr']):
                    self.set_lr(opt, decayed_lr)
            else:
                self.set_lr(optimizers, decayed_lr)

        # Increment the step counter
        self.state['steps'] += 1

    # Helper method to set the learning rate for the optimizer(s)
    def set_lr(self, optimizer, lr):
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

    def load_state_dict(self, state_dict):
        self.state.update(state_dict)

    def state_dict(self):
        return self.state.copy()

### Model Weight Averaging
This is to average the weights of multiple models by averaging them to improve the overall performance here.

In [ ]:
def average_models(models):
    """Averages model weights and loads the resulting weights into the first model, returning it"""
    models = list(models)
    n = len(models)
    sds = [m.state_dict() for m in models]
    averaged = {}
    for k in sds[0]:
        averaged[k] = sum(sd[k] for sd in sds) / n
    models[0].load_state_dict(averaged)
    return models[0]

## Main

This is where the main execution happens, from data pre-processing to training, to (in the future) validation as well.

This code includes the testing as well. As part of the final project, I will abstract this into a separate module completely.

In [ ]:
seed = 2020
pl.seed_everything(seed)
dm = physionet.PhysioNetDataModule(batch_size=512, num_workers=16, use_temp_cache=True)
dm.setup()
pretrain_model = duett.pretrain_model(d_static_num=dm.d_static_num(),
        d_time_series_num=dm.d_time_series_num(), d_target=dm.d_target(), pos_frac=dm.pos_frac(),
        seed=seed)
checkpoint = pl.callbacks.ModelCheckpoint(save_last=True, monitor='val_loss', mode='min', save_top_k=1, dirpath='checkpoints')
warmup = WarmUpCallback(steps=2000)
trainer = pl.Trainer(gpus=1, logger=False, num_sanity_val_steps=2, max_epochs=300,
        gradient_clip_val=1.0, callbacks=[warmup, checkpoint])
trainer.fit(pretrain_model, dm)

pretrained_path = checkpoint.best_model_path
for seed in range(2020, 2023):
    pl.seed_everything(seed)
    fine_tune_model = duett.fine_tune_model(pretrained_path, d_static_num=dm.d_static_num(),
            d_time_series_num=dm.d_time_series_num(), d_target=dm.d_target(), pos_frac=dm.pos_frac(), seed=seed)
    checkpoint = pl.callbacks.ModelCheckpoint(save_top_k=5, save_last=False, mode='max', monitor='val_ap', dirpath='checkpoints')
    warmup = WarmUpCallback(steps=1000)
    trainer = pl.Trainer(gpus=1, logger=False, max_epochs=50, gradient_clip_val=1.0,
            callbacks=[warmup, checkpoint])
    trainer.fit(fine_tune_model, dm)
    final_model = average_models([duett.fine_tune_model(path, d_static_num=dm.d_static_num(),
            d_time_series_num=dm.d_time_series_num(), d_target=dm.d_target(), pos_frac=dm.pos_frac())
            for path in checkpoint.best_k_models.keys()])
    trainer.test(final_model, dataloaders=dm)

# Results
Will be filled in final report.


## Model comparison
Will be filled in final report.


# Discussion
Will be filled in final report.




# References

1.   Labach, A., Pokhrel, A., Huang, X. S., Zuberi, S., Yi, S. E., Volkovs, M., ... & Krishnan, R. G. (2023, December). Duett: Dual event time transformer for electronic health records. In Machine Learning for Healthcare Conference (pp. 403-422). PMLR.
2. Johnson, A., Bulgarelli, L., Pollard, T., Horng, S., Celi, L. A., & Mark, R. (2023). MIMIC-IV Clinical Database Demo (version 2.2). PhysioNet. https://doi.org/10.13026/dp1f-ex47.